Notebook to perform a hyperparameter sweep for the customizable model.
We also will be using the TF dependent adjustment function to closer mimic the real data.

In [1]:
# imports
import argparse
from argparse import Namespace

from pytorch_lightning import Trainer, LightningModule, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger
from torchsummary import summary

from yeastdnnexplorer.data_loaders.synthetic_data_loader import SyntheticDataLoader
from yeastdnnexplorer.ml_models.simple_model import SimpleModel
from yeastdnnexplorer.ml_models.customizable_model import CustomizableModel

import optuna

import matplotlib.pyplot as plt
import seaborn as sns

from yeastdnnexplorer.probability_models.generate_data import (
    perturbation_effect_adjustment_function_with_tf_relationships,
)

seed_everything(42)

Seed set to 42


42

In [2]:
# define checkpoints for the model
# tells it when to save snapshots of the model during training
# Callback to save the best model based on validation loss
best_model_checkpoint = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    filename="best-model-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,
)

# Callback to save checkpoints every 5 epochs, regardless of performance
periodic_checkpoint = ModelCheckpoint(
    filename="periodic-{epoch:02d}",
    every_n_epochs=2,
    save_top_k=-1,  # Setting -1 saves all checkpoints
)

# define loggers for the model# configure loggers
tb_logger = TensorBoardLogger("logs/tensorboard_logs")
csv_logger = CSVLogger("logs/csv_logs")

Old scratch work [PROOF that a good model is learning nonlinear deps introduced by tf_relationships_dict]

In [3]:
tf_relationships_dict = {
    0: [2, 4, 7],
    1: [8],
    2: [3, 9],
    3: [1, 6],
    4: [5],
    5: [0, 2, 8],
    6: [4],
    7: [1, 4],
    8: [6],
    9: [0, 3, 8],
}

data_module = SyntheticDataLoader(
    batch_size=32,
    num_genes=4000,
    signal_mean=3.0,
    signal=[0.5] * 10,  # old: [0.1, 0.15, 0.2, 0.25, 0.3],
    n_sample=[1, 2, 2, 4, 4],  # sum of this is num of tfs
    val_size=0.1,
    test_size=0.1,
    random_state=42,
    max_mean_adjustment=3.0,
    adjustment_function=perturbation_effect_adjustment_function_with_tf_relationships,
    tf_relationships=tf_relationships_dict,
)

num_tfs = sum(data_module.n_sample)  # sum of all n_sample is the number of TFs

model = CustomizableModel(
    input_dim=num_tfs,
    output_dim=num_tfs,
    lr=0.01,
    hidden_layer_num=3,
    hidden_layer_sizes=[128, 64, 32],
    activation="ReLU",
    optimizer="Adam",
    L2_regularization_term=0.0,
    dropout_rate=0.0,
)

trainer = Trainer(
    max_epochs=10,
    deterministic=True,
    accelerator="cpu",
    callbacks=[best_model_checkpoint, periodic_checkpoint],
    logger=[tb_logger, csv_logger],
)

trainer.fit(model, data_module)

test_results = trainer.test(model, datamodule=data_module)
print("Printing test results...")
print(test_results)  # this prints all metrics that were logged during the test phase

# print summary of model
print("Printing model summary...")
summary(model, (num_tfs, num_tfs))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/yeastdnnexplorer/data_loaders/synthetic_data_loader.py:244: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train, Y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(
/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/yeastdnnexplorer/data_loaders/synthetic_data_loader.py:247: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val, Y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(
/Users/benmueller/2024Classes/BrentResearch/git

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           1.7528094053268433
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Printing test results...
[{'test_loss': 1.7528094053268433}]
Printing model summary...
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 13, 128]           1,792
              ReLU-2              [-1, 13, 128]               0
           Dropout-3              [-1, 13, 128]               0
            Linear-4               [-1, 13, 64]           8,256
              ReLU-5               [-1, 13, 64]               0
          

Define tf relationships dictionary

In [3]:
tf_relationships_dict = {
    0: [2, 4, 7],
    1: [8],
    2: [3, 9],
    3: [1, 6],
    4: [5],
    5: [0, 2, 8],
    6: [4],
    7: [1, 4],
    8: [6],
    9: [0, 3, 8],
}

Define objective function for our hyperparam sweep (and decide on what we want to sweep through)

In [5]:
# hyperparams
# input_dim: int,
# output_dim: int,
# lr: float = 0.001,
# hidden_layer_num: int = 1,
# hidden_layer_sizes: list = [128],
# activation: str = "ReLU", # can be "ReLU", "Sigmoid", "Tanh", "LeakyReLU"
# optimizer: str = "Adam", # can be "Adam", "SGD", "RMSprop"
# L2_regularization_term: float = 0.0,
# dropout_rate: float = 0.0,


def objective(trial):
    # model hyperparameters
    lr = trial.suggest_categorical("lr", [1e-4, 1e-3, 1e-2, 1e-1])
    hidden_layer_num = trial.suggest_categorical("hidden_layer_num", [1, 2, 3, 5])
    activation = trial.suggest_categorical(
        "activation", ["ReLU", "Sigmoid", "Tanh", "LeakyReLU"]
    )
    optimizer = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])
    L2_regularization_term = trial.suggest_categorical(
        "L2_regularization_term", [0, 0.1, 0.01]
    )  # change to categorical?
    dropout_rate = trial.suggest_categorical(
        "dropout_rate", [0, 0.3, 0.5]
    )  # change to categorical?

    # data module hyperparameters
    batch_size = trial.suggest_categorical("batch_size", [32, 128])

    # training hyperparameters
    max_epochs = trial.suggest_categorical(
        "max_epochs", [2]
    )  # can keep this low for sanity check

    # defining what to pass in for the hidden layer sizes list based on the number of hidden layers
    hidden_layer_sizes_configurations = {
        1: [[64], [256]],
        2: [[64, 32], [256, 64]],
        3: [[256, 128, 32]],
        5: [[512, 256, 128, 64, 32]],
    }
    hidden_layer_sizes = trial.suggest_categorical(
        f"hidden_layer_sizes_{hidden_layer_num}_layers",
        hidden_layer_sizes_configurations[hidden_layer_num],
    )

    print("=" * 70)
    print("About to create model with the following hyperparameters:")
    print(f"lr: {lr}")
    print(f"hidden_layer_num: {hidden_layer_num}")
    print(f"hidden_layer_sizes: {hidden_layer_sizes}")
    print(f"activation: {activation}")
    print(f"optimizer: {optimizer}")
    print(f"L2_regularization_term: {L2_regularization_term}")
    print(f"dropout_rate: {dropout_rate}")
    print(f"batch_size: {batch_size}")
    print(f"max_epochs: {max_epochs}")
    print("")

    # create data module
    data_module = SyntheticDataLoader(
        batch_size=batch_size,
        num_genes=4000,
        signal_mean=3.0,
        signal=[0.5] * 10,  # old: [0.1, 0.15, 0.2, 0.25, 0.3],
        n_sample=[1, 2, 2, 4, 4],  # sum of this is num of tfs
        val_size=0.1,
        test_size=0.1,
        random_state=42,
        max_mean_adjustment=3.0,
        adjustment_function=perturbation_effect_adjustment_function_with_tf_relationships,
        tf_relationships=tf_relationships_dict,
    )

    num_tfs = sum(data_module.n_sample)  # sum of all n_sample is the number of TFs

    # create model
    model = CustomizableModel(
        input_dim=num_tfs,
        output_dim=num_tfs,
        lr=lr,
        hidden_layer_num=hidden_layer_num,
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        optimizer=optimizer,
        L2_regularization_term=L2_regularization_term,
        dropout_rate=dropout_rate,
    )

    # create trainer
    trainer = Trainer(
        max_epochs=max_epochs,
        deterministic=True,
        accelerator="cpu",
        callbacks=[
            best_model_checkpoint
        ],  # not using periodic checkpoint as that would be way too many checkpoints, can add back if we choose a specific hyperparam config that we want to look more closely at
        logger=[tb_logger, csv_logger],
    )

    # train model
    trainer.fit(model, data_module)

    # get best validation loss from the model
    return trainer.callback_metrics["val_loss"]  # TODO is .item() needed?

Now we run the hyperparam sweep.

In [6]:
# Perform hyperparameter optimization using Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

# Get the best hyperparameters and their corresponding values
best_params = study.best_params
best_loss = study.best_value

print("\n" * 5)
print("RESULTS" + ("=" * 70))
print(f"Best hyperparameters: {best_params}")
print(f"Best loss: {best_loss}")

[I 2024-03-18 10:30:40,965] A new study created in memory with name: no-name-a994b30b-aa3d-4830-8de9-152b9aaaab76
/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/.venv/lib/python3.11/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [64] which is of type list.
  warnings.warn(message)
/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/.venv/lib/python3.11/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [256] which is of type list.
  warnings.warn(message)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


About to create model with the following hyperparameters:
lr: 0.001
hidden_layer_num: 1
hidden_layer_sizes: [64]
activation: ReLU
optimizer: Adam
L2_regularization_term: 0.01
dropout_rate: 0.5
batch_size: 32
max_epochs: 2

bm - adjustment function provided to dataLoader setup


/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/yeastdnnexplorer/data_loaders/synthetic_data_loader.py:231: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train, Y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(
/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/yeastdnnexplorer/data_loaders/synthetic_data_loader.py:234: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val, Y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(
/Users/benmueller/2024Classes/BrentResearch/git_repos/yeastdnnexplorer/yeastdnnexplorer/data_loaders/synthetic_data_loader.py:237: UserWarning: To copy construct from a tensor, it is recommended

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
[I 2024-03-18 10:32:32,349] Trial 4 finished with value: 2.3827872276306152 and parameters: {'lr': 0.01, 'hidden_layer_num': 5, 'activation': 'Tanh', 'optimizer': 'RMSprop', 'L2_regularization_term': 0.1, 'dropout_rate': 0.3, 'batch_size': 128, 'max_epochs': 2, 'hidden_layer_sizes_5_layers': [512, 256, 128, 64, 32]}. Best is trial 4 with value: 2.3827872276306152.








RESULTS======================================================================
Best hyperparameters: {'lr': 0.01, 'hidden_layer_num': 5, 'activation': 'Tanh', 'optimizer': 'RMSprop', 'L2_regularization_term': 0.1, 'dropout_rate': 0.3, 'batch_size': 128, 'max_epochs': 2, 'hidden_layer_sizes_5_layers': [512, 256, 128, 64, 32]}
Best loss: 2.3827872276306152
